In [1]:
#install various libraries
#pip install kiteconnect
#pip install jugaad-trader

In [2]:
#https://marketsetup.in/documentation/jugaad-trader/cli/

In [3]:
#error: if jtrader is not found, try reinstalling the jugaad-trader pip

In [6]:
#run this in command in terminal
#jtrader zerodha startsession

In [7]:
import datetime
import pandas as pd
import logging
from kiteconnect import KiteConnect
logging.basicConfig(level=logging.DEBUG)
from jugaad_trader import Zerodha

In [8]:
kite = Zerodha()
# Set access token loads the stored session.
# Name chosen to keep it compatible with kiteconnect.
kite.set_access_token() # loads the session from config folder
profile = kite.profile()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): kite.zerodha.com:443
DEBUG:urllib3.connectionpool:https://kite.zerodha.com:443 "GET /oms/user/profile HTTP/1.1" 200 None


In [5]:
#now run this command in terminal
#jtrader zerodha savecreds

In [6]:
#the above command will save the credentials in a app config folder in file named .zcred

In [7]:
kite = Zerodha()
kite.load_creds()
kite.login()
#get profile
#print(kite.profile())

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): kite.zerodha.com:443
DEBUG:urllib3.connectionpool:https://kite.zerodha.com:443 "GET / HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://kite.zerodha.com:443 "POST /api/login HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://kite.zerodha.com:443 "POST /api/twofa HTTP/1.1" 200 42


{'status': 'success', 'data': {'profile': {}}}

In [8]:
# Get margin
margins = kite.margins()
#print(margins)

DEBUG:urllib3.connectionpool:https://kite.zerodha.com:443 "GET /oms/user/margins HTTP/1.1" 200 None


In [9]:
# Get holdings
holdings = kite.holdings()
#print(holdings)

DEBUG:urllib3.connectionpool:https://kite.zerodha.com:443 "GET /oms/portfolio/holdings HTTP/1.1" 200 None


In [10]:
# Get today's positions
positions = kite.positions()
#print(positions)

DEBUG:urllib3.connectionpool:https://kite.zerodha.com:443 "GET /oms/portfolio/positions HTTP/1.1" 200 None


In [11]:
# Get today's orders
orders = kite.orders()
print(orders)

DEBUG:urllib3.connectionpool:https://kite.zerodha.com:443 "GET /oms/orders HTTP/1.1" 200 None


[{'placed_by': 'ZG4424', 'order_id': '200727101982614', 'exchange_order_id': None, 'parent_order_id': None, 'status': 'REJECTED', 'status_message': 'This exchange segment is not enabled for your account. Enable by visiting: zrd.sh/add-segment', 'status_message_raw': 'exchg not enabled for this acct', 'order_timestamp': datetime.datetime(2020, 7, 27, 20, 37, 12), 'exchange_update_timestamp': None, 'exchange_timestamp': None, 'variety': 'regular', 'exchange': 'MCX', 'tradingsymbol': 'SILVERMIC20AUGFUT', 'instrument_token': 55655943, 'order_type': 'LIMIT', 'transaction_type': 'BUY', 'validity': 'DAY', 'product': 'NRML', 'quantity': 1, 'disclosed_quantity': 0, 'price': 65256, 'trigger_price': 0, 'average_price': 0, 'filled_quantity': 0, 'pending_quantity': 0, 'cancelled_quantity': 0, 'market_protection': 0, 'meta': {}, 'tag': None, 'guid': '19Xbnsrxblifhjg'}]


In [7]:
#https://kite.trade/docs/pykiteconnect/v3/#kiteconnect.KiteConnect.historical_data
def historical_data(instrument_token, from_date, to_date, interval, continuous=False, oi=False):
    """
    Retrieve historical data (candles) for an instrument.
    Although the actual response JSON from the API does not have field
    names such has 'open', 'high' etc., this function call structures
    the data into an array of objects with field names. For example:
    - `instrument_token` is the instrument identifier (retrieved from the instruments()) call.
    - `from_date` is the From date (datetime object or string in format of yyyy-mm-dd HH:MM:SS.
    - `to_date` is the To date (datetime object or string in format of yyyy-mm-dd HH:MM:SS).
    - `interval` is the candle interval (minute, day, 5 minute etc.).
    - `continuous` is a boolean flag to get continuous data for futures and options instruments.
    - `oi` is a boolean flag to get open interest.
    """
    date_string_format = "%Y-%m-%d %H:%M:%S"
    from_date_string = from_date.strftime(date_string_format) if type(from_date) == datetime.datetime else from_date
    to_date_string = to_date.strftime(date_string_format) if type(to_date) == datetime.datetime else to_date
    data = _get("market.historical", {
        "instrument_token": instrument_token,
        "from": from_date_string,
        "to": to_date_string,
        "interval": interval,
        "continuous": 1 if continuous else 0,
        "oi": 1 if oi else 0
    })
    return _format_historical(data, oi)

In [80]:
instrument_token = 738561
from_date = '2015-01-01 09:15:00'
to_date = '2015-02-28 15:30:00'
interval = 'minute'

In [81]:
get_data = kite.historical_data(instrument_token,from_date,to_date,interval,)

DEBUG:urllib3.connectionpool:https://kite.zerodha.com:443 "GET /oms/instruments/historical/738561/minute?instrument_token=738561&from=2015-01-01+09%3A15%3A00&to=2015-02-28+15%3A30%3A00&interval=minute&continuous=0&oi=0 HTTP/1.1" 200 118847


In [82]:
data = pd.DataFrame(get_data)

In [83]:
data

,date,open,high,low,close,volume
0,2015-02-02 09:15:00+05:30,458.48,459.90,458.18,458.40,108278
1,2015-02-02 09:16:00+05:30,458.40,458.68,457.50,457.50,46142
2,2015-02-02 09:17:00+05:30,457.55,458.40,457.40,458.00,48890
3,2015-02-02 09:18:00+05:30,458.08,459.25,458.08,458.75,56594
4,2015-02-02 09:19:00+05:30,458.73,458.75,458.25,458.43,31682
...,...,...,...,...,...,...
7495,2015-02-28 15:25:00+05:30,433.40,433.40,432.78,433.00,17800
7496,2015-02-28 15:26:00+05:30,433.50,433.90,433.28,433.48,16810
7497,2015-02-28 15:27:00+05:30,433.45,433.88,433.40,433.85,16688
7498,2015-02-28 15:28:00+05:30,433.90,433.90,433.15,433.63,14664


In [14]:
instruments = pd.read_csv('https://api.kite.trade/instruments')

In [69]:
instruments

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
0,544844550,2128299,EURINR20AUG79.2500CE,EURINR,0,2020-08-27,79.25,0.0025,1,CE,BCD-OPT,BCD
1,544838918,2128277,EURINR20AUG79.2500PE,EURINR,0,2020-08-27,79.25,0.0025,1,PE,BCD-OPT,BCD
2,544788742,2128081,EURINR20AUG79.5000CE,EURINR,0,2020-08-27,79.50,0.0025,1,CE,BCD-OPT,BCD
3,544782342,2128056,EURINR20AUG79.5000PE,EURINR,0,2020-08-27,79.50,0.0025,1,PE,BCD-OPT,BCD
4,544739078,2127887,EURINR20AUG79.7500CE,EURINR,0,2020-08-27,79.75,0.0025,1,CE,BCD-OPT,BCD
...,...,...,...,...,...,...,...,...,...,...,...,...
67272,1945089,7598,ZODJRDMKJ-BE,ZODIAC JRD MKJ-,0,NaN,0.00,0.0500,1,EQ,NSE,NSE
67273,2916865,11394,ZOTA,ZOTA HEALTH CARE,0,NaN,0.00,0.0500,1,EQ,NSE,NSE
67274,7436801,29050,ZUARI,ZUARI AGRO CHEMICALS,0,NaN,0.00,0.0500,1,EQ,NSE,NSE
67275,979713,3827,ZUARIGLOB,ZUARI GLOBAL,0,NaN,0.00,0.0500,1,EQ,NSE,NSE


In [16]:
instruments[instruments['tradingsymbol']=='RELIANCE']

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
14464,128083204,500325,RELIANCE,RELIANCE INDUSTRIES.,0,NaN,0.0,0.05,1,EQ,BSE,BSE
66618,738561,2885,RELIANCE,RELIANCE INDUSTRIES,0,NaN,0.0,0.05,1,EQ,NSE,NSE
